## This notebook trains different models to predict newborn weight

In [3]:
from datetime import datetime

import pickle
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

from pandas_profiling import ProfileReport

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import RandomizedSearchCV

import tensorflow as tf
from childbirth_common_util import *


In [4]:
print(datetime.now())

2023-02-08 22:08:43.747318


In [11]:
# laod CSV data

X_train_from_file = pd.read_csv("data_files/x_train_orig.csv")
y_train_from_file = pd.read_csv("data_files/y_train_orig.csv")
y_train_weight = y_train_from_file['birth_weight_in_g'].squeeze()
y_train_age = y_train_from_file['combined_gestation_week'].squeeze()

X_val_from_file = pd.read_csv("data_files/x_val_orig.csv")
y_val_from_file = pd.read_csv("data_files/y_val_orig.csv")
y_val_weight = y_val_from_file['birth_weight_in_g'].squeeze()
y_val_age = y_val_from_file['combined_gestation_week'].squeeze()

X_test_from_file = pd.read_csv("data_files/x_test_orig.csv")
y_test_from_file = pd.read_csv("data_files/y_test_orig.csv")
y_test_weight = y_test_from_file['birth_weight_in_g'].squeeze()
y_test_age = y_test_from_file['combined_gestation_week'].squeeze()

print(f"Train feature shape: {X_train_from_file.shape}, output shape: {y_train_weight.shape}")
print(f"Val feature shape: {X_val_from_file.shape}, output shape: {y_val_weight.shape}")
print(f"Test feature shape: {X_test_from_file.shape}, output shape: {y_test_weight.shape}")


Train feature shape: (116615, 81), output shape: (116615,)
Val feature shape: (38872, 81), output shape: (38872,)
Test feature shape: (38872, 81), output shape: (38872,)


In [12]:
util_calc_baseline(y_train_weight, "birth_weight_in_g")

the birth_weight_in_g's mean in training is 3249.156257771299
birth_weight_in_g: rmse=588.133500299761


In [13]:
# list of regressor model: 
# https://scikit-learn.org/stable/supervised_learning.html

# Train Weight Models


In [14]:
# load feature list from file
column_list = np.loadtxt(f'models/feature_list_weight.txt', dtype="object")

print(column_list)
print(column_list.shape)


['birth_month' 'mother_age' 'mother_nativity' 'residence_status'
 'mother_race1' 'mother_hispanic_race' 'paternity_acknowledged'
 'marital_status' 'mother_education' 'father_age'
 'prior_births_now_living' 'total_birth_order'
 'interval_since_last_live_birth' 'month_prenatal_care_began'
 'number_of_prenatal_visits' 'wic' 'cigarettes_3rd_trimester'
 'mother_height_in_total_inches' 'bmi' 'prepregnancy_weight'
 'weight_gain_group' 'gestational_diabetes' 'prepregnancy_hypertension'
 'gestational_hypertension' 'previous_preterm_birth'
 'infertility_treatment_used' 'fertility_enhancing_drugs'
 'previous_cesarean' 'number_of_previous_cesareans'
 'no_risk_factors_reported' 'chlamydia' 'attendant_at_birth' 'pluarality'
 'sex_of_infant' 'last_normal_menses_month' 'combined_gestation_week'
 'birth_weight_in_g' 'infant_breastfed_at_discharge']
(38,)


In [15]:
# calculate and saved the scaler for each feature
X_train_scaled = util_handle_na(X_train_from_file[column_list].copy())
X_train_scaled = util_calc_save_scaler(X_train_scaled, "weight")
print(X_train_scaled.shape)

# sanity check. mean = 0, std dev = 1
display(X_train_scaled.describe())

(116615, 36)


,birth_month,mother_age,mother_nativity,residence_status,mother_race1,mother_hispanic_race,paternity_acknowledged,marital_status,mother_education,father_age,...,fertility_enhancing_drugs,previous_cesarean,number_of_previous_cesareans,no_risk_factors_reported,chlamydia,attendant_at_birth,pluarality,sex_of_infant,last_normal_menses_month,infant_breastfed_at_discharge
count,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,...,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05,1.166150e+05
mean,2.935383e-15,-5.743627e-16,2.133346e-15,6.035561e-16,1.366370e-17,-7.564606e-16,1.522374e-15,3.427292e-16,1.446023e-15,4.644343e-17,...,-1.765016e-15,-2.038079e-15,4.748392e-16,8.206749e-16,1.715268e-15,4.287309e-16,-5.123558e-16,-1.196499e-15,-2.143769e-16,-1.516061e-15
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,...,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,-1.603606e+00,-3.315649e+00,-4.285933e+00,-4.911274e+00,-2.860839e+00,-1.819003e+00,-1.916987e+00,-1.267869e+00,-1.383852e+00,-2.610609e+00,...,-9.867132e+00,-3.179457e+00,-6.533818e+00,-3.468656e+00,-1.093861e+01,-1.417491e+00,-1.610491e+01,-1.021016e+00,-1.739787e+00,-2.163216e+00
25%,-7.380822e-01,-7.017324e-01,5.190669e-01,-1.198648e+00,5.483675e-01,-2.976907e-01,-9.933853e-01,-1.267869e+00,-1.040109e+00,-6.253568e-01,...,1.363072e-01,-4.193374e-01,-3.379821e-01,-1.374388e+00,1.426313e-01,-5.093799e-01,1.791575e-01,-1.021016e+00,-9.088700e-01,-7.986342e-01
50%,1.274418e-01,2.940454e-01,5.190669e-01,6.576646e-01,5.483675e-01,8.432934e-01,8.538175e-01,8.946347e-01,-3.526239e-01,3.992893e-01,...,1.363072e-01,-4.193374e-01,-3.379821e-01,7.198793e-01,1.426313e-01,-5.093799e-01,1.791575e-01,9.794169e-01,-7.795316e-02,5.659479e-01
75%,9.929658e-01,7.919344e-01,5.190669e-01,6.576646e-01,5.483675e-01,8.432934e-01,8.538175e-01,8.946347e-01,1.022346e+00,7.835316e-01,...,1.363072e-01,-4.193374e-01,-3.379821e-01,7.198793e-01,1.426313e-01,-5.093799e-01,1.791575e-01,9.794169e-01,7.529637e-01,5.659479e-01
max,1.569982e+00,1.289823e+00,5.190669e-01,6.576646e-01,1.478151e+00,8.432934e-01,8.538175e-01,8.946347e-01,1.366089e+00,1.295855e+00,...,1.363072e-01,2.340782e+00,2.759936e+00,7.198793e-01,1.426313e-01,3.123063e+00,1.791575e-01,9.794169e-01,1.583881e+00,5.659479e-01


In [16]:
# scaled the data to mean=0 and std dev=1
X_val_scaled = util_scale(X_val_from_file[column_list], 'weight')
print(X_val_scaled.shape)

# sanity check. mean = 0, std dev = 1
display(X_val_scaled.describe())

(38872, 36)


,birth_month,mother_age,mother_nativity,residence_status,mother_race1,mother_hispanic_race,paternity_acknowledged,marital_status,mother_education,father_age,...,fertility_enhancing_drugs,previous_cesarean,number_of_previous_cesareans,no_risk_factors_reported,chlamydia,attendant_at_birth,pluarality,sex_of_infant,last_normal_menses_month,infant_breastfed_at_discharge
count,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,...,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000
mean,-0.009895,-0.005992,-0.007577,-0.001061,0.000544,-0.001193,-0.000333,-0.000920,-0.008943,-0.002804,...,-0.004631,-0.008288,-0.006095,0.002413,0.010644,-0.001756,-0.016057,0.003491,-0.002411,0.011578
std,0.995752,1.003704,1.007157,0.999944,0.998502,0.999211,0.999710,1.000435,1.000126,0.997011,...,1.009901,0.994746,0.995315,1.000800,0.962873,0.998421,1.035900,0.999934,0.997114,0.990842
min,-1.603606,-3.315649,-4.285933,-4.911274,-2.705875,-1.819003,-1.916987,-1.267869,-1.383852,-2.610609,...,-9.867132,-3.179457,-5.501179,-3.468656,-10.938614,-1.417491,-16.104906,-1.021016,-1.739787,-2.163216
25%,-0.738082,-0.701732,0.519067,-1.198648,0.548367,-0.297691,-0.993385,-1.267869,-1.040109,-0.625357,...,0.136307,-0.419337,-0.337982,-1.374388,0.142631,-0.509380,0.179158,-1.021016,-0.908870,-0.798634
50%,0.127442,0.294045,0.519067,0.657665,0.548367,0.843293,0.853817,0.894635,-0.352624,0.399289,...,0.136307,-0.419337,-0.337982,0.719879,0.142631,-0.509380,0.179158,0.979417,-0.077953,0.565948
75%,0.704458,0.791934,0.519067,0.657665,0.548367,0.843293,0.853817,0.894635,1.022346,0.783532,...,0.136307,-0.419337,-0.337982,0.719879,0.142631,-0.509380,0.179158,0.979417,0.752964,0.565948
max,1.569982,1.289823,0.519067,0.657665,1.323187,0.843293,0.853817,0.894635,1.366089,1.167774,...,0.136307,2.340782,2.759936,0.719879,0.142631,3.123063,0.179158,0.979417,1.583881,0.565948


## Train individual model and evaluate its accuracy

In [26]:
# train individual models and use validation dataset to compare the results
# It helps determine the proportion of each model for ensemble modeling
# Please refer to childbirth_model_parameter_tuning.ipynb for parameters tuning code
def train_and_evaluate_several_models_for_weight_prediction(predict_output_type, X_train, y_train, X_val, y_val):

    # linear regression
    model = LinearRegression() # rmse=515(30K), 504(200K)
    util_train_and_evaluate("linear", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # Gradient Boosting Regressor, rmse=493(30K), 468(200K)
    params = {"n_estimators": 500, "max_depth": 4, "min_samples_split": 5, "learning_rate": 0.05, "loss": "squared_error"}
    model = GradientBoostingRegressor(**params)
    util_train_and_evaluate("gb", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # SGDRegressor, rmse=515(30K), 503(200K)
    params = {'penalty': 'l2', 'loss': 'squared_error', 'learning_rate': 'adaptive', 'eta0': 10, 'alpha': 0.01}
    model = SGDRegressor(**params)
    util_train_and_evaluate("sgd", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # LGBMRegressor, rmse=504(30K), 476(200K)
    model = LGBMRegressor()
    util_train_and_evaluate("lgbm", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # XGBRegressor, rmse=505(30K), 461(200K)
    model = XGBRegressor()
    util_train_and_evaluate("xgb", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # RandomForestRegressor, 500(200K)
    # it takes 10+ minutes
    #params = {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 100, 'bootstrap': True}
    params = {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 10, 'bootstrap': True}
    model = RandomForestRegressor(**params)
    util_train_and_evaluate("rf", predict_output_type, model, X_train, y_train, X_val, y_val)
    print("")

    # SVR is slow and bad, Dont Use.Take 30 minutes to run
    # SVR, rmse = 517(30K), 516(200K). 515(200K). 
    #params = {'kernel': 'linear', 'gamma': 1e-07, 'epsilon': 0.1, 'degree': 2, 'coef0': 1, 'C': 100, 'max_iter': 150000}}
    #model = SVR(**params)
    #train_and_evaluate("svr", predict_output_type, model, X_train, y_train, X_val, y_val)
    #print("")
    
    # KNN is very bad. Don't use
    #model = KNeighborsRegressor(n_neighbors=20)
    #train_and_evaluate("KNN", model, X_train_from_file, y_train_weight, X_val_from_file, y_val_weight)
    #print("")



In [28]:
# it takes over 30 minutes to traing and evaluate the above models
train_and_evaluate_several_models_for_weight_prediction("weight", X_train_scaled, y_train_weight, X_val_scaled, y_val_weight)


Start training model linear for weight at 2023-02-08 22:45:46.641903
Saving linear to file: models/model_linear_weight.sav
End time = 2023-02-08 22:45:46.900904, elapsed time = 0.2590007781982422
linear for weight: rmse=503.9419764008061

Start training model gb for weight at 2023-02-08 22:45:46.900904
Saving gb to file: models/model_gb_weight.sav
End time = 2023-02-08 22:47:54.711035, elapsed time = 127.8101315498352
gb for weight: rmse=468.36134200607626

Start training model sgd for weight at 2023-02-08 22:47:54.712035
Saving sgd to file: models/model_sgd_weight.sav
End time = 2023-02-08 22:47:57.420008, elapsed time = 2.707973003387451
sgd for weight: rmse=503.9449797194499

Start training model lgbm for weight at 2023-02-08 22:47:57.420008
Saving lgbm to file: models/model_lgbm_weight.sav
End time = 2023-02-08 22:47:58.110374, elapsed time = 0.690366268157959
lgbm for weight: rmse=476.4496511079465

Start training model xgb for weight at 2023-02-08 22:47:58.110374
Saving xgb to fi

In [21]:
# Neural network, rmse=505 (200K)
def nn_train_and_evaluate_for_weight_prediction(predict_output_type, X_train, y_train, X_val, y_val):
    model_name="nn"
    start_time = time.time()
    print(f"Start training model {model_name} for {predict_output_type} at {datetime.now()}")

    tf.keras.backend.clear_session()
    tf.random.set_seed(0)

    nn_model = tf.keras.Sequential()
    nn_model.add(tf.keras.layers.Dense(16))
    nn_model.add(tf.keras.layers.Dense(units=1))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

    # We specify the MSE loss.
    nn_model.compile(loss='mse', optimizer=optimizer)
    history = nn_model.fit(
      x = X_train,
      y = y_train,
      epochs=100,
      batch_size=32,
      validation_split=0.2,
      verbose=0)

    y_pred = nn_model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)

    model_filename = f"models/model_{model_name}_{predict_output_type}"
    print(f"Saving {model_name} to directory: {model_filename}")
    
    nn_model.save(model_filename)
    end_time = time.time()
    print(f"End time = {datetime.now()}, elapsed time = {end_time - start_time}")
    
    print(f"{model_name} for {predict_output_type}: rmse={rmse}")
    

nn_train_and_evaluate_for_weight_prediction("weight", X_train_scaled, y_train_weight, X_val_scaled, y_val_weight)


Start training model nn for weight at 2023-02-08 22:27:42.948279
Saving nn to directory: models/model_nn_weight
INFO:tensorflow:Assets written to: models/model_nn_weight\assets
End time = 2023-02-08 22:32:35.018072, elapsed time = 292.0697937011719
nn for weight: rmse=505.6505345734763


# Ensemble Models for Weight Prediction

In [29]:
# Ensemble model for weight prediction

column_list = util_load_x_columns_list_from_file("weight")
models = util_load_models_from_file("weight")

def scale_predict_compare_save(X_input_from_file, y, input_type):
    predict_output_type = "weight"
    X_scaled = util_scale(X_input_from_file[column_list], predict_output_type)
    print(f"{input_type} feature shape: {X_scaled.shape}, output shape: {y.shape}")
    y_pred = util_ensemble_predict_weight(X_scaled, column_list, models)
    print(y_pred)
    print(y_pred.shape)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    print(rmse) 
    np.savetxt(f"data_files/pred_y_{input_type}_{predict_output_type}.csv", y_pred, delimiter=",")

# rmse=469
scale_predict_compare_save(X_input_from_file = X_val_from_file, y = y_val_weight, input_type = "val")

# rmse=454
scale_predict_compare_save(X_input_from_file = X_train_from_file, y = y_train_weight, input_type = "train")

# rmse=472
scale_predict_compare_save(X_input_from_file = X_test_from_file, y = y_test_weight, input_type = "test")


val feature shape: (38872, 36), output shape: (38872,)
predicting using linear and its proportion is 0.05
predicting using gb and its proportion is 0.25
predicting using sgd and its proportion is 0.05
predicting using lgbm and its proportion is 0.25
predicting using xgb and its proportion is 0.25
predicting using rf and its proportion is 0.05
predicting using nn and its proportion is 0.1
[3275.41444125 3433.30016533 2517.94526429 ... 3376.01292692 3393.04876656
 2556.39956524]
(38872,)
469.45291029417405
train feature shape: (116615, 36), output shape: (116615,)
predicting using linear and its proportion is 0.05
predicting using gb and its proportion is 0.25
predicting using sgd and its proportion is 0.05
predicting using lgbm and its proportion is 0.25
predicting using xgb and its proportion is 0.25
predicting using rf and its proportion is 0.05
predicting using nn and its proportion is 0.1
[3514.09801294 3224.85259173 3195.35093767 ... 2756.28292162 3074.37601837
 3024.94113924]
(116